In [1]:
from src.dataloading import *

dataset = CustomAudioDataset(data_dir= "/import/research_c4dm/JulienMarcoChrisRMRI/MTAT_wav", target_length=5, n_augmentations=2)

/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

test_data = dataset[0]

In [5]:
test_data['wav'].shape

torch.Size([2, 1, 120000])

In [5]:
import torch
def get_positive_mask(batch_size, seq_length):
        # Create a mask where positives are indicated by consecutive elements in each batch
        pos_mask = torch.zeros(batch_size, seq_length, dtype=torch.bool)
        for i in range(batch_size):
            start_idx = i * seq_length
            end_idx = (i + 1) * seq_length
            pos_mask[i, start_idx:end_idx] = True

        # Extend the positive window for local tokens
        # for i in range(seq_length):
        #     start_pos = max(0, i - 3 // 2)
        #     end_pos = min(seq_length, i + 3 // 2 + 1)
        #     pos_mask[i, start_pos:end_pos] = True

        return pos_mask
    
print(get_positive_mask(8,10))

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False]])


In [35]:
import torch
import torch.nn as nn

class SupConLoss(nn.Module):
    """Supervised Contrastive Learning: https://arxiv.org/pdf/2004.11362.pdf.
    It also supports the unsupervised contrastive loss in SimCLR"""
    def __init__(self, temperature=0.07, contrast_mode='all',
                 base_temperature=0.07):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode
        self.base_temperature = base_temperature

    def forward(self, features, labels=None, mask=None):
        """Compute loss for model. If both `labels` and `mask` are None,
        it degenerates to SimCLR unsupervised loss:
        https://arxiv.org/pdf/2002.05709.pdf

        Args:
            features: hidden vector of shape [bsz, n_views, ...].
            labels: ground truth of shape [bsz].
            mask: contrastive mask of shape [bsz, bsz], mask_{i,j}=1 if sample j
                has the same class as sample i. Can be asymmetric.
        Returns:
            A loss scalar.
        """
        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('cpu'))

        if len(features.shape) < 3:
            raise ValueError('`features` needs to be [bsz, n_views, ...],'
                             'at least 3 dimensions are required')
        if len(features.shape) > 3:
            features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]
        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32).to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float().to(device)
        else:
            mask = mask.float().to(device)

        print(mask)

        contrast_count = features.shape[1]
        
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        print(contrast_feature.shape)
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))

        # compute logits
        anchor_dot_contrast = torch.div(
            torch.matmul(anchor_feature, contrast_feature.T),
            self.temperature)
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()
        
        print(logits.shape)

        # tile mask
        mask = mask.repeat(anchor_count, contrast_count) ## what is this mask?
        print(mask.shape)
        # mask-out self-contrast cases
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1).to(device),
            0
        ) ## is this only self-contrast?
        
        mask = mask * logits_mask
        
        print(mask)

        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))
        
        print(log_prob.shape)

        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

        # loss
        loss = - (self.temperature / self.base_temperature) * mean_log_prob_pos
        # loss = loss.view(anchor_count, batch_size)

        return loss.mean()
    
    
loss = SupConLoss()
test_data = torch.zeros((6,1,512))
test_mask = torch.zeros(3,3)
test_mask[0,1] = 1
test_mask[1,0] = 1
test_mask[1,1] = 1

test_labels = torch.Tensor([1,1,1,2,2,1])
loss(test_data, test_labels)

tensor([[1., 1., 1., 0., 0., 1.],
        [1., 1., 1., 0., 0., 1.],
        [1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 1., 1., 0.],
        [1., 1., 1., 0., 0., 1.]])
torch.Size([6, 512])
torch.Size([6, 6])
torch.Size([6, 6])
tensor([[0., 1., 1., 0., 0., 1.],
        [1., 0., 1., 0., 0., 1.],
        [1., 1., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0.],
        [1., 1., 1., 0., 0., 0.]])
torch.Size([6, 6])


tensor(1.6094)

In [40]:
torch.cat(torch.split(torch.zeros((1,12000)),12000)).unsqueeze(1).shape

torch.Size([1, 1, 12000])

In [17]:
# def get_contrastive_matrix(batch_size, sequence_length, num_augmentations, window_size):
#     total_samples = batch_size* sequence_length * num_augmentations
#     same_sample = torch.zeros((total_samples, total_samples), dtype=torch.int)
#     window_mask = torch.zeros_like(same_sample)
    
#     window_width = window_size//2 +1

#     for i in range(total_samples):
#         i_indices = []
#         for j in range(total_samples):
#             if i // sequence_length == j // sequence_length:
#                 i_indices.append(j)
#         same_sample[i,i_indices] = 1
    
#     # return contrastive_matrix
    
#     for k in range(-window_width, window_width):
#         window_mask += torch.diag(torch.ones((total_samples)),k)[:total_samples,:total_samples].int()
    
#     return same_sample * window_mask

import torch

def get_contrastive_matrix(batch_size, sequence_length, num_augmentations, window_size):
    total_samples = batch_size * sequence_length * num_augmentations

    # Create a matrix indicating whether samples are from the same sequence
    
    diag_block = torch.ones((sequence_length,sequence_length))
    same_sequence = torch.block_diag(*((batch_size * num_augmentations) * [diag_block]))
    

    # Create a window mask
    window_width = window_size // 2 + 1
    window_mask = torch.diag(torch.ones((total_samples)), 0)

    for k in range(1, window_width):
        window_mask += torch.diag(torch.ones((total_samples - k)), k)
        window_mask += torch.diag(torch.ones((total_samples - k)), -k)

    # Apply the window mask
    window_mask = window_mask[:total_samples, :total_samples].int()
    

    # Return the contrastive matrix
    return same_sequence * window_mask

print(get_contrastive_matrix(2,6,1,10))

tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]], dtype=torch.int32)
tensor([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0

In [61]:
torch.diag(torch.ones(3),1)[:3,:3]

tensor([[0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 0.]])